# Indicadores



### Instalando e importando bibliotecas

In [15]:
%pip install yfinance # Para capturar os preços
%pip install ta # Para calcular os indicadores
%pip install plotly # Para visualização dos indicadores


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
# Importe as bibliotecas
import ta
import plotly.graph_objects as go
import yfinance as yf
import pandas as pd
from datetime import date

### Capturando preços

In [18]:
def get_price_tickers_br(tickers, start_date, end_date):
    df = pd.DataFrame()
    
    start_date_search = start_date
    end_date_search   = end_date
    
    
    ticker = str(tickers) + '.SA'
    print(f'Baixando os dados de {ticker}')
    
    df = yf.download(ticker, 
                        start = start_date_search, 
                        end   = end_date_search)

    print('Download completo')
    return df


start_date = '2022-01-01'
end_date   = date.today()     
tickers = 'PETR4'
df = get_price_tickers_br(tickers, start_date, end_date)
df

Baixando os dados de PETR4.SA
[*********************100%%**********************]  1 of 1 completed
Download completo


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,28.540001,29.219999,28.530001,29.090000,14.063067,52704700
2022-01-04,29.160000,29.400000,28.910000,29.200001,14.116246,51739200
2022-01-05,29.190001,29.270000,27.940001,28.070000,13.569967,78459800
2022-01-06,28.290001,28.650000,27.840000,28.049999,13.560297,61163100
2022-01-07,28.110001,28.290001,27.820000,28.180000,13.623144,47507600
...,...,...,...,...,...,...
2024-03-22,35.900002,36.220001,35.430000,36.049999,36.049999,34426800
2024-03-25,36.180000,36.830002,36.160000,36.599998,36.599998,26951500
2024-03-26,36.669998,36.669998,36.169998,36.259998,36.259998,24837800


## Indicadores de Momentum

### RSI - Índice de força relativa

Este indicadore mede o tamanho das variações de preços para saber se a ação está **sobrecomprada ou sobrevendida**

A variação deste indicador é de 0 a 100. Tradicionalmente, **acima de 70** pode indicar que a ação está próximo de está sobrecomprada, ou seja, em breve o preço deve começar a cair. E **abaixo de 30** pode indicar que o valor está sobrevendido e o preço tende voltar a subir.

A representação matemática deste indicador é:
$$
RSI = 100 - \frac{100}{1 + RS}
$$

Onde:

$$
RS = \frac{\text{Média das últimas subidas de preços}}{\text{Média das últimas quedas de preços}}
$$

Desta forma, você consegue gerar o indice de 0 a 100 e entender se o preço está sobrecomprado ou sobrevendido.

In [17]:
from ta.momentum import RSIIndicator
from plotly.subplots import make_subplots

In [20]:
rsi = RSIIndicator(close=df['Close'], window=30)
df['RSI'] = rsi.rsi()

In [62]:
# Gráfico preços / RSI

fig = make_subplots(rows = 2, cols = 1)

fig.add_trace(go.Scatter(name = 'RSI',
                         x    = df.index,
                         y    = df['RSI']),
                         row  = 2,
                         col  = 1
                         )

fig.add_trace(go.Candlestick(name  = 'PETR4',
                             x     = df.index,
                             close = df['Close'],
                             open  = df['Open'],
                             high  = df['High'],
                             low   = df['Low']),
                             row   = 1,
                             col   = 1
                             )

fig.update_xaxes(rangeslider_visible=False)

fig.show()

### PPO - Percentage Price Oscillator

O PPO demonstra a relção entre 2 médias móveis exponenciais em termos percentuais, e é usada para confirmar sinais de reversão de tendências levando em consideração a volatilidade e a performance do ativo.

A representação matemática deste indicador é: 
$$
PPO = \frac{{(EMA_{curto prazo} - EMA_{longo prazo})}}{{EMA_{longo prazo}}} \times 100
$$

Onde:
$$
EMA_{curto prazo} = representa a Média Móvel Exponencial de curto prazo.
$$
$$
EMA_{longo prazo} = representa a Média Móvel Exponencial de longo prazo.
$$
Neste caso, consideraremos como MME de curto prazo a média de 12 períodos, e para longo prazo 26 períodos.

Este indicador funciona de maneira semelhante ao indicador MACD, onde, quando há o cruzamento entre a linha de PPO com o sinal deste indicador, indica uma reversão de tendência.


In [63]:
from ta.momentum import PercentagePriceOscillator

In [64]:
PPO = PercentagePriceOscillator(df['Close'])

df['PPO']      = PPO.ppo()        # Valor do indicador
df['PPO_hist'] = PPO.ppo_hist()   # Intensidade do volume
df['PPO_line'] = PPO.ppo_signal() # Sinal do PPO

In [68]:
fig = make_subplots(rows  = 2,
                    cols  = 1,
                    specs = [[{'secondary_y':True}],[{'secondary_y':True}]])

fig.add_trace(go.Scatter(name = 'PPO',
                         x    = df.index,
                         y    = df['PPO']),
                         row  = 2,
                         col  = 1)

fig.add_trace(go.Bar(name        = 'PPO_hist',
                     x           = df.index,
                     y           = df['PPO_hist']),
                     row         = 2,
                     col         = 1,
                     secondary_y = True)

fig.add_trace(go.Scatter(name = 'PPO_line',
                         x    = df.index,
                         y    = df['PPO_line']),
                         row  = 2,
                         col  = 1)

fig.add_trace(go.Candlestick(name  = 'PETR4',
                             x     = df.index,
                             close = df['Close'],
                             open  = df['Open'],
                             high  = df['High'],
                             low   = df['Low']),
                             row   = 1,
                             col   = 1)

fig.update_xaxes(rangeslider_visible = False)

fig.show()

### Rate of change

O Rate of Change (ROC) é um indicador técnico que mede a variação percentual do preço ao longo de um período específico. 

Este indicador oscila acima e abaixo de zero. Ele simplesmente compara o preço de agora com o preço de N períodos anteriores para indicar se o mercado está sobrecomprado ou sobrevendido.

A representação matemática deste indicador é:
$$
ROC = \frac{{Preço_{atual} - Preço_{anterior}}}{{Preço_{anterior}}} \times 100
$$

Onde:
$$
Preço_{atual} = é o preço atual do ativo.
$$
$$
Preço_{anterior} = é o preço do ativo no período anterior.
$$

